<h1>Week 3 Assignment</h1>

<h4>Import Numpy and Pandas</h4>

In [19]:
import numpy as np
import pandas as pd

<h4>Read data from the Wiki page using Pandas</h4>

In [23]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
print(df.shape)
df.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h4>Drop rows that don't have Borough</h4>

In [24]:
df.drop(df.index[df['Borough'] == "Not assigned"], inplace = True)
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h4>Check if there are any "Not assigned" Neighbourhood</h4>

In [27]:
print("Number of Not assigned Neighbourhood: " + str(df_raw[df_raw['Neighbourhood'] == "Not assigned"].shape[0]))

Number of Not assigned Neighbourhood: 0


<h4>There is no "Not assigned" Neighbourhood, so we don't need to perform any clean-up</h4>

<h4>Check to see if there are any duplicated Postal Code</h4>

In [30]:
dup = df[df.duplicated(['Postal Code'])]
print(dup)

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


<h4>There is no duplicated Postal Code so no more clean-up is needed</h4>

<h4>So here is the final number of rows of the data frame:</h4>

In [31]:
df.shape[0]

103